In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import copy
from skimage import io as skio
from skimage.transform import rescale
# My own modules
import sys
import dp_numpy

In [2]:
for name in ['barbara','peppers_bw']:
    f = np.load('images/'+name+'_noisy.npy')
    M,N = f.shape[0:2]
    white = np.ones([M,N])
    plt.imsave('results/images_paper/'+name+'_white.png',white,vmin=0,vmax=1,cmap='gray')

# Compute Belief Propagation Ground Truths

In [72]:
reg_par_list = [10,15,20,30,50]
reg_par_list = [20,30]
tau_list = [1e-5,1e-6]

for name in ['barbara','peppers_bw']:
    for reg_num,reg_par in enumerate(reg_par_list):
    
        ##################################
        ##################################
        # Belief Propagation
        f = np.load('images/'+name+'_noisy.npy')
        M,N = f.shape[0:2]
        
        # save white image as a spacer for figures in paper
        white = np.ones([M,N])
        plt.imsave('results/images_paper/'+name+'_white.png',white,vmin=0,vmax=1,cmap='gray')

        sigma = 0.05
        K = 100
        labels=np.linspace(0,1,K)
        unaries = (f[:,:,np.newaxis] - labels[np.newaxis,np.newaxis,:])**2/(2*sigma**2)

        ##################################
        # Pairwise
        k,l = np.meshgrid(labels, labels)

        # Truncated TV
        T = 1
        lamb = reg_par
        pairwise = np.clip(np.abs(k-l), a_min = None, a_max = T) *lamb
        w = None
        NB = None


        # Sweep BP: Horizontal+Vertical
        one_hot,map_labels,marginals = dp_numpy.sbp(unaries, pairwise, w, NB, maxit=10, softmin=True, verbose=1)

        # compute normalized marginal distributions
        mm = -marginals.copy()
        mm -= mm.max(axis=-1, keepdims=True)
        mm = np.exp(mm)
        mm /= np.sum(mm, axis=-1, keepdims=True)

        # compute MMSE
        u_bp = np.sum(mm*labels[np.newaxis,np.newaxis,:], axis=-1)

        # compute pixel variance
        var_bp = np.sum(mm*(labels[np.newaxis,np.newaxis,:]-u_bp[:,:,None])**2, axis=-1)
        
        np.save('results/image_ex/denoising/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_belief_propagation_mmse.npy',u_bp)
        np.save('results/image_ex/denoising/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_tau_belief_propagation_variance.npy',var_bp)
        
            

Pairwise cost: FULL
iter =  0 , time =  5.96633 , E =  32841.38921
iter =  1 , time =  11.89900 , E =  31934.71702
iter =  2 , time =  17.82784 , E =  31910.52453
iter =  3 , time =  23.75651 , E =  31891.03208
iter =  4 , time =  29.68343 , E =  31888.58267
iter =  5 , time =  35.61386 , E =  31888.98838
iter =  6 , time =  41.54537 , E =  31889.98993
iter =  7 , time =  47.47459 , E =  31890.58064
iter =  8 , time =  53.40512 , E =  31890.58064
iter =  9 , time =  59.33519 , E =  31890.58064
Pairwise cost: FULL
iter =  0 , time =  5.93683 , E =  39689.43136
iter =  1 , time =  11.86522 , E =  38195.81905
iter =  2 , time =  17.79153 , E =  38052.94727
iter =  3 , time =  23.71621 , E =  38006.41976
iter =  4 , time =  29.63768 , E =  37998.16547
iter =  5 , time =  35.56268 , E =  37987.60160
iter =  6 , time =  41.48917 , E =  37987.67565
iter =  7 , time =  47.41756 , E =  37987.22203
iter =  8 , time =  53.34434 , E =  37987.30250
iter =  9 , time =  59.27225 , E =  37987.38232
Pa

# Graphs Comparing BP with Proposed for Denoising for all Step Sizes and Reg Parameters

In [31]:
reg_par_list = [30]
error_dict = {}
niter = 100000
step = 500

tau_list = [1e-5,1e-6]


for method in ['prox_subgrad','grad_subgrad']:
    error_dict_method = {}
    for name in ['barbara','peppers_bw']:
        err = np.zeros([len(reg_par_list),len(tau_list),round(niter//step)+1])
        for reg_num,reg_par in enumerate(reg_par_list):
            

            u_bp = np.load('results/image_ex/denoising/'+name+'_reg_par_'+str(reg_par)+
                           '_data_par_399,99999999999994_belief_propagation_mmse.npy')
            var_bp = np.load('results/image_ex/denoising/'+name+'_reg_par_'+str(reg_par)+
                             '_data_par_399,99999999999994_tau_belief_propagation_variance.npy')

            plt.imsave('results/images_paper/image_denoising/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_belief_propagation_mmse.png',
                        u_bp,vmin=0,vmax=1,cmap='gray')
            plt.imsave('results/images_paper/image_denoising/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_belief_propagation_variance.png',
                        var_bp,cmap='hot',vmin=0.0004,vmax = 0.0026)

            for tau_num,tau in enumerate(tau_list):

                u_langevin = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_mmse.npy').squeeze()
                var_langevin = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance.npy').squeeze()
                x0 = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_tau_'+str(tau)+'_x0.npy').squeeze()

                plt.imsave('results/images_paper/image_denoising/'+method+'/'+name+'_x0.png',
                          x0.squeeze(),vmin=0,vmax=1,cmap='gray')

                plt.imsave('results/images_paper/image_denoising/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_mmse.png',
                          u_langevin.squeeze(),vmin=0,vmax=1,cmap='gray')
                
                
                f = plt.figure(frameon=False)
                f = plt.imshow(var_langevin.squeeze(),cmap='hot',extent = (0,1,0,1),vmin=0.0004,vmax = 0.0026)
                plt.colorbar(f)
                plt.tick_params(which = 'both', size = 0, labelsize = 0)
                plt.savefig('results/images_paper/image_denoising/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance_colorbar.png')
                plt.close('all')
                plt.imsave('results/images_paper/image_denoising/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance.png',
                          var_langevin.squeeze(),cmap='hot',vmin=0.0004,vmax = 0.0026)

                for r,i in enumerate(range(1000000,int(1e6+niter+1),step)):
                    u_langevin = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_tau_'+str(tau)+'_iter_'+str(i)+'_mmse.npy').squeeze()

                    err[reg_num,tau_num,r] = np.sum(np.square(u_langevin-u_bp))

        error_dict_method[name] = np.copy(err)
    error_dict[method] = copy.deepcopy(error_dict_method)





In [26]:
ld = '0.0001'


for method in ['MYULA']:
    error_dict_method = {}
    for name in ['barbara','peppers_bw']:
        err = np.zeros([len(reg_par_list),len(tau_list),round(niter//step)+1])
        for reg_num,reg_par in enumerate(reg_par_list):
            

            u_bp = np.load('results/image_ex/denoising/'+name+'_reg_par_'+str(reg_par)+
                           '_data_par_399,99999999999994_belief_propagation_mmse.npy')
            var_bp = np.load('results/image_ex/denoising/'+name+'_reg_par_'+str(reg_par)+
                             '_data_par_399,99999999999994_tau_belief_propagation_variance.npy')

            for tau_num,tau in enumerate(tau_list):
                u_langevin = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_mmse.npy').squeeze()
                var_langevin = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance.npy').squeeze()
                x0 = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_tau_'+str(tau)+'_x0.npy').squeeze()

                plt.imsave('results/images_paper/image_denoising/'+method+'/'+name+'_x0.png',
                          x0.squeeze(),vmin=0,vmax=1,cmap='gray')

                plt.imsave('results/images_paper/image_denoising/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_mmse.png',
                          u_langevin.squeeze(),vmin=0,vmax=1,cmap='gray')
                
                
                f = plt.figure(frameon=False)
                f = plt.imshow(var_langevin.squeeze(),cmap='hot',extent = (0,1,0,1),vmin=0.0004,vmax = 0.0026)
                plt.colorbar(f)
                plt.tick_params(which = 'both', size = 0, labelsize = 0)
                plt.savefig('results/images_paper/image_denoising/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance_colorbar.png')
                plt.close('all')
                plt.imsave('results/images_paper/image_denoising/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_399,99999999999994_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance.png',
                          var_langevin.squeeze(),cmap='hot',vmin=0.0004,vmax = 0.0026)

                for r,i in enumerate(range(1000000,int(1e6+niter+1),step)):
                    u_langevin = np.load('results/image_ex/denoising/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_399.99999999999994_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(i)+'_mmse.npy').squeeze()

                    err[reg_num,tau_num,r] = np.sum(np.square(u_langevin-u_bp))

        error_dict_method[name] = np.copy(err)
    error_dict[method] = copy.deepcopy(error_dict_method)





In [27]:
iterations = np.array(list(range(0,int(niter+1),500)))
c = ['b', 'g','r' , 'k']

for name in ['barbara','peppers_bw']:
    for reg_num,reg_par in enumerate(reg_par_list):
        plt.figure(figsize=(20,15))
        for tau_num,tau in enumerate(tau_list):
            plt.plot(iterations,np.log10(error_dict['grad_subgrad'][name][reg_num,tau_num]),c[tau_num],alpha = 0.3,linewidth=10,label=r'$\tau=$'+str(tau)+'\n'+'Grad-Sub')
            plt.plot(iterations,np.log10(error_dict['prox_subgrad'][name][reg_num,tau_num]),c[tau_num],linewidth=2,label='Prox-Sub')
            plt.plot(iterations,np.log10(error_dict['MYULA'][name][reg_num,tau_num]),c[tau_num]+'--',linewidth=2,label=r'MYULA $\theta = $'+str(ld))
    
        matplotlib.rcParams.update({'font.size': 20})
        plt.xlabel('Iteration $k$',fontsize=30)
        plt.ylabel(r'$log_{10}(||\bar{x}^N_k - \bar{x}||^2_2)$',fontsize=30)
        plt.legend()
        plt.savefig('results/images_paper/image_denoising/'+name+'_reg_par'+str(reg_par)+'.png')
        plt.close('all')

# For Deconvolution

In [48]:
c = ['b', 'g','r' , 'k']
error_dict = {}
tau_list=[1e-5,1e-6]
reg_par_list = [20]
niter = 500000
step = 5000

for method in ['grad_subgrad','prox_subgrad']:
    error_dict_method = {}
    for name in ['peppers_bw','barbara']:
        if name == 'barbara':
            vmax = 0.0031
            vmin = 0.0005
        else:
            vmax = 0.011
            vmin = 0.0001
        err = np.zeros([len(reg_par_list),len(tau_list),round(niter//step)+1])
        for reg_num,reg_par in enumerate(reg_par_list):

            u_metro = np.load('results/image_ex/deconv_kernelsize_5/grad_subgrad/metro/'+name+'/reg_par_'+str(reg_par)+
                           '_data_par_10000.0_tau_1e-06_iter_4000000_mmse.npy').squeeze()
            var_metro = np.load('results/image_ex/deconv_kernelsize_5/grad_subgrad/metro/'+name+'/reg_par_'+str(reg_par)+
                           '_data_par_10000.0_tau_1e-06_iter_4000000_variance.npy').squeeze()

            plt.imsave('results/images_paper/image_deconvolution/'+name+'_reg_par_'+str(reg_par)+
                           '_data_par_10000.0_tau_1e-06_iter_4000000_mmse.png',u_metro,vmin=0,vmax=1,cmap='gray')
            plt.imsave('results/images_paper/image_deconvolution/'+name+'_reg_par_'+str(reg_par)+
                           '_data_par_10000.0_tau_1e-06_iter_4000000_variance.png',var_metro,cmap='hot',vmin=vmin,vmax=vmax)

            for tau_num,tau in enumerate(tau_list):

                u_langevin = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_iter_'+str(int(1e6+niter))+'_mmse.npy').squeeze()
                var_langevin = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_iter_'+str(int(1e6+niter))+'_variance.npy').squeeze()
                x0 = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_x0.npy').squeeze()
                plt.imsave('results/images_paper/image_deconvolution/'+method+'/'+name+'_x0.png',
                          x0.squeeze(),cmap='gray')

                plt.imsave('results/images_paper/image_deconvolution/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_iter_'+str(int(1e6+niter))+'_mmse.png',
                          u_langevin.squeeze(),vmin=0,vmax=1,cmap='gray')
                plt.imsave('results/images_paper/image_deconvolution/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_iter_'+str(int(1e6+niter))+'_variance.png',
                          var_langevin.squeeze(),cmap='hot',vmin=vmin,vmax=vmax)
                
                f = plt.figure(frameon=False)
                f = plt.imshow(var_langevin.squeeze(),cmap='hot',extent = (0,1,0,1),vmin=vmin,vmax=vmax)
                plt.colorbar(f)
                plt.tick_params(which = 'both', size = 0, labelsize = 0)
                plt.savefig('results/images_paper/image_deconvolution/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_iter_'+str(int(1e6+niter))+'_variance_colorbar.png')
                plt.close('all')

                for r,i in enumerate(range(1000000,int(1e6+niter+1),step)):
                    u_langevin = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_iter_'+str(i)+'_mmse.npy').squeeze()
                    err[reg_num,tau_num,r] = np.sum(np.square(u_langevin-u_metro))

        error_dict_method[name] = np.copy(err)
    error_dict[method] = copy.deepcopy(error_dict_method)



In [49]:
ld = '0.0001'

for method in ['MYULA']:
    error_dict_method = {}
    for name in ['barbara','peppers_bw']:
        if name == 'barbara':
            vmax = 0.0031
            vmin = 0.0005
        else:
            vmax = 0.011
            vmin = 0.0001
        err = np.zeros([len(reg_par_list),len(tau_list),round(niter//step)+1])
        for reg_num,reg_par in enumerate(reg_par_list):
            

            u_metro = np.load('results/image_ex/deconv_kernelsize_5/grad_subgrad/metro/'+name+'/reg_par_'+str(reg_par)+
                           '_data_par_10000.0_tau_1e-06_iter_4000000_mmse.npy').squeeze()
            var_metro = np.load('results/image_ex/deconv_kernelsize_5/grad_subgrad/metro/'+name+'/reg_par_'+str(reg_par)+
                           '_data_par_10000.0_tau_1e-06_iter_4000000_variance.npy').squeeze()

            for tau_num,tau in enumerate(tau_list):
                u_langevin = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_mmse.npy').squeeze()
                var_langevin = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance.npy').squeeze()
                x0 = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_tau_'+str(tau)+'_x0.npy').squeeze()

                plt.imsave('results/images_paper/image_deconvolution/'+method+'/'+name+'_x0.png',
                          x0.squeeze(),vmin=0,vmax=1,cmap='gray')

                plt.imsave('results/images_paper/image_deconvolution/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_10000.0_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_mmse.png',
                          u_langevin.squeeze(),vmin=0,vmax=1,cmap='gray')
                
                
                f = plt.figure(frameon=False)
                f = plt.imshow(var_langevin.squeeze(),cmap='hot',extent = (0,1,0,1),vmin=vmin,vmax=vmax)
                plt.colorbar(f)
                plt.tick_params(which = 'both', size = 0, labelsize = 0)
                plt.savefig('results/images_paper/image_deconvolution/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_10000.0_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance_colorbar.png')
                plt.close('all')
                plt.imsave('results/images_paper/image_deconvolution/'+method+'/'+name+'_reg_par_'+str(reg_par)+'_data_par_10000.0_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(int(niter+1e6))+'_variance.png',
                          var_langevin.squeeze(),cmap='hot',vmin=vmin,vmax=vmax)

                for r,i in enumerate(range(1000000,int(1e6+niter+1),step)):
                    u_langevin = np.load('results/image_ex/deconv_kernelsize_5/'+method+'/'+name+'/reg_par_'+str(reg_par)+'_data_par_10000.0_ld_'+ld+'_tau_'+str(tau)+'_iter_'+str(i)+'_mmse.npy').squeeze()

                    err[reg_num,tau_num,r] = np.sum(np.square(u_langevin-u_metro))

        error_dict_method[name] = np.copy(err)
    error_dict[method] = copy.deepcopy(error_dict_method)





In [50]:
iterations = np.array(list(range(0,int(niter+1),step)))

for name in ['peppers_bw']:
    for reg_num,reg_par in enumerate(reg_par_list):
        plt.figure(figsize=(20,15))
        for tau_num,tau in enumerate(tau_list):
            plt.plot(iterations,np.log10(error_dict['grad_subgrad'][name][reg_num,tau_num]),c[tau_num],alpha = 0.3,linewidth=10,label=r'$\tau=$'+str(tau)+'\n'+'Grad-Sub')
            plt.plot(iterations,np.log10(error_dict['prox_subgrad'][name][reg_num,tau_num]),c[tau_num],linewidth=2,label='Prox-Sub')
            plt.plot(iterations,np.log10(error_dict['MYULA'][name][reg_num,tau_num]),c[tau_num]+'--',linewidth=2,label=r'MYULA $\theta = $'+str(ld))
    
        matplotlib.rcParams.update({'font.size': 20})
        plt.xlabel('Iteration $k$',fontsize=30)
        plt.ylabel(r'$log_{10}(||\bar{x}^N_k - \bar{x}||^2_2)$',fontsize=30)
        plt.legend()
        plt.savefig('results/images_paper/image_deconvolution/'+name+'_reg_par'+str(reg_par)+'.png')
        plt.close('all')

In [138]:
t = np.load('results/image_ex/deconv_kernelsize_5/grad_subgrad/metro/barbara/reg_par_20_data_par_10000.0_tau_1e-06_computation_times.npy')

In [139]:
print(np.sum(t>0))


445
